# Introduction

In this document, I will try to have fun with the google btree. I
will try to find a a spot where simd instructions can be inserted with
a hope of gain; afterwards, let's benchmark some new version(s) and
find out why the compiler is so smart (compared to me at least).

# Description of the test environment

Here are some details on the bench machine:

* CPU: Intel Core i5-6200U @ 2.30 GHz
* RAM: 16GB
* System: Arch (linux-4.6.4, glibc-2.33)
* Cpufreq governor: performance: the pstate power scaling driver is
  used; so the CPU frequency cannot be statically set with a
  user-space tool.
```
# cpupower frequency-set -g performance
```

# Finding SIMD-friendly code

A btree is composed of nodes (really?) and these nodes may have an
interesting property: they could hold many tuples "key - value" in a
contiguous memory area; so, why not trying to have a look at the
lookup code.

```
  // Returns the position of the first value whose key is not less than k using
  // linear search performed using plain compare.
  template <typename Compare>
  int linear_search_plain_compare(
      const key_type &k, int s, int e, const Compare &comp) const {
    __asm__ __volatile__ ("nop; nop; nop; nop; nop"); // ### MARK ###
    while (s < e) {
      if (!btree_compare_keys(comp, key(s), k)) {
        break;
      }
      ++s;
    }
    __asm__ __volatile__ ("nop; nop; nop; nop; nop"); // ### MARK ###
    return s;
  }
```

Calling the function btree_compare_keys() means calling std::less (or
std::greater); so this call should be replaced by basic comparison
instruction in the end. Let's have a look at the disassembled code:

```
  405e97:	90                   	nop
  405e98:	90                   	nop
  405e99:	90                   	nop
  405e9a:	90                   	nop
  405e9b:	90                   	nop
  405e9c:	85 d2                	test   %edx,%edx
  405e9e:	0f 84 6c 03 00 00    	je     406210 <_Z25bench_no_unroll_btree_mapv+0x4d0>
  405ea4:	48 8b 0e             	mov    (%rsi),%rcx
  405ea7:	48 3b 48 10          	cmp    0x10(%rax),%rcx
  405eab:	0f 86 5f 03 00 00    	jbe    406210 <_Z25bench_no_unroll_btree_mapv+0x4d0>
  405eb1:	83 fa 01             	cmp    $0x1,%edx
  405eb4:	0f 84 46 03 00 00    	je     406200 <_Z25bench_no_unroll_btree_mapv+0x4c0>
  405eba:	48 39 48 20          	cmp    %rcx,0x20(%rax)
  405ebe:	0f 83 3c 03 00 00    	jae    406200 <_Z25bench_no_unroll_btree_mapv+0x4c0>
  405ec4:	83 fa 02             	cmp    $0x2,%edx
  405ec7:	0f 84 23 03 00 00    	je     4061f0 <_Z25bench_no_unroll_btree_mapv+0x4b0>
  405ecd:	48 3b 48 30          	cmp    0x30(%rax),%rcx
  405ed1:	0f 86 19 03 00 00    	jbe    4061f0 <_Z25bench_no_unroll_btree_mapv+0x4b0>
  405ed7:	83 fa 03             	cmp    $0x3,%edx
  405eda:	0f 84 00 03 00 00    	je     4061e0 <_Z25bench_no_unroll_btree_mapv+0x4a0>
  405ee0:	48 3b 48 40          	cmp    0x40(%rax),%rcx
  405ee4:	0f 86 f6 02 00 00    	jbe    4061e0 <_Z25bench_no_unroll_btree_mapv+0x4a0>
  405eea:	83 fa 04             	cmp    $0x4,%edx
  405eed:	0f 84 dd 02 00 00    	je     4061d0 <_Z25bench_no_unroll_btree_mapv+0x490>
  405ef3:	48 3b 48 50          	cmp    0x50(%rax),%rcx
  405ef7:	0f 86 d3 02 00 00    	jbe    4061d0 <_Z25bench_no_unroll_btree_mapv+0x490>
  405efd:	83 fa 05             	cmp    $0x5,%edx
  405f00:	0f 84 ba 02 00 00    	je     4061c0 <_Z25bench_no_unroll_btree_mapv+0x480>
  405f06:	48 3b 48 60          	cmp    0x60(%rax),%rcx
  405f0a:	0f 86 b0 02 00 00    	jbe    4061c0 <_Z25bench_no_unroll_btree_mapv+0x480>
  405f10:	83 fa 06             	cmp    $0x6,%edx
  405f13:	0f 84 97 02 00 00    	je     4061b0 <_Z25bench_no_unroll_btree_mapv+0x470>
  405f19:	48 39 48 70          	cmp    %rcx,0x70(%rax)
  405f1d:	0f 83 8d 02 00 00    	jae    4061b0 <_Z25bench_no_unroll_btree_mapv+0x470>
  405f23:	83 fa 07             	cmp    $0x7,%edx
  405f26:	0f 84 74 02 00 00    	je     4061a0 <_Z25bench_no_unroll_btree_mapv+0x460>
  405f2c:	48 3b 88 80 00 00 00 	cmp    0x80(%rax),%rcx
  405f33:	0f 86 67 02 00 00    	jbe    4061a0 <_Z25bench_no_unroll_btree_mapv+0x460>
  405f39:	83 fa 08             	cmp    $0x8,%edx
  405f3c:	0f 84 4e 02 00 00    	je     406190 <_Z25bench_no_unroll_btree_mapv+0x450>
  405f42:	48 3b 88 90 00 00 00 	cmp    0x90(%rax),%rcx
  405f49:	0f 86 41 02 00 00    	jbe    406190 <_Z25bench_no_unroll_btree_mapv+0x450>
  405f4f:	83 fa 09             	cmp    $0x9,%edx
  405f52:	0f 84 28 02 00 00    	je     406180 <_Z25bench_no_unroll_btree_mapv+0x440>
  405f58:	48 3b 88 a0 00 00 00 	cmp    0xa0(%rax),%rcx
  405f5f:	0f 86 1b 02 00 00    	jbe    406180 <_Z25bench_no_unroll_btree_mapv+0x440>
  405f65:	83 fa 0a             	cmp    $0xa,%edx
  405f68:	0f 84 02 02 00 00    	je     406170 <_Z25bench_no_unroll_btree_mapv+0x430>
  405f6e:	48 3b 88 b0 00 00 00 	cmp    0xb0(%rax),%rcx
  405f75:	0f 86 f5 01 00 00    	jbe    406170 <_Z25bench_no_unroll_btree_mapv+0x430>
  405f7b:	83 fa 0b             	cmp    $0xb,%edx
  405f7e:	0f 84 dc 01 00 00    	je     406160 <_Z25bench_no_unroll_btree_mapv+0x420>
  405f84:	48 3b 88 c0 00 00 00 	cmp    0xc0(%rax),%rcx
  405f8b:	0f 86 cf 01 00 00    	jbe    406160 <_Z25bench_no_unroll_btree_mapv+0x420>
  405f91:	83 fa 0c             	cmp    $0xc,%edx
  405f94:	0f 84 b6 01 00 00    	je     406150 <_Z25bench_no_unroll_btree_mapv+0x410>
  405f9a:	48 3b 88 d0 00 00 00 	cmp    0xd0(%rax),%rcx
  405fa1:	0f 86 a9 01 00 00    	jbe    406150 <_Z25bench_no_unroll_btree_mapv+0x410>
  405fa7:	83 fa 0d             	cmp    $0xd,%edx
  405faa:	0f 84 90 01 00 00    	je     406140 <_Z25bench_no_unroll_btree_mapv+0x400>
  405fb0:	48 3b 88 e0 00 00 00 	cmp    0xe0(%rax),%rcx
  405fb7:	0f 86 83 01 00 00    	jbe    406140 <_Z25bench_no_unroll_btree_mapv+0x400>
  405fbd:	83 fa 0e             	cmp    $0xe,%edx
  405fc0:	0f 84 6a 02 00 00    	je     406230 <_Z25bench_no_unroll_btree_mapv+0x4f0>
  405fc6:	48 3b 88 f0 00 00 00 	cmp    0xf0(%rax),%rcx
  405fcd:	0f 97 c1             	seta   %cl
  405fd0:	0f b6 c9             	movzbl %cl,%ecx
  405fd3:	83 c1 0e             	add    $0xe,%ecx
  405fd6:	90                   	nop
  405fd7:	90                   	nop
  405fd8:	90                   	nop
  405fd9:	90                   	nop
  405fda:	90                   	nop
```

The loop has been kind of unrolled; there is no index increment;
according to the count of key items in the node, the code will be
executed until the end or not:

* The register edx contains the count of elements in the node;
* Let's take an example: there are 5 elements; so once, the
  instruction pointer reaches the address 0x405efd; the instruction
  "je" (jump if equal) will end the further comparisons;

# Trying to prevent unrolling

In order to save instruction decoding operations, 2 micro-ops cache
were inserted right after the instruction decoder:

* Starting from Nehalem, a loop buffer was added so as to hold the
  micro-ops executed in a iteration of a loop;
* Starting from SandyBridge, a micro-ops cache is available to get rid
  of the limitation of 16 bytes per clock cycle in the fetch / decode
  unit;

Considering these improvements wihtin the processor pipeline, it may
be interesting to prevent the loop-unrolling we saw in the assembly
code.

## Code

I was unable to disable the specific "-f..." option in gcc which enable
the specific unrolling optimization; I tried to disable one by one all
the optimization features of the "O3" (either with "#pragma GCC
optimize" or directly with arguments "-fno-..."optimization basket but
it did not work;

But disabling the whole "O3" optimizations set worked; here is the
code and the corresponding assembly.

```
#pragma GCC push_options
#pragma GCC optimize ("O2")
  
  // Returns the position of the first value whose key is not less than k using
  // linear search performed using plain compare.
  template <typename Compare>
  int linear_search_plain_compare(
      const key_type &k, int s, int e, const Compare &comp) const {

      __asm__ __volatile__ ("nop; nop; nop; nop; nop");
      
    while (s < e) {
      if (!btree_compare_keys(comp, key(s), k)) {
        break;
      }
      ++s;
    }

    __asm__ __volatile__ ("nop; nop; nop; nop; nop");
    
    return s;
  }
#pragma GCC pop_options
```

Once more the reader will note the inserted nop so as to locate the
assembly code we are interested in.

## Assembly code:

```
0000000000402610 <_ZNK15no_unroll_btree10btree_nodeINS_16btree_map_paramsImmSt4lessImESaISt4pairIKmmEELi256EEEE27linear_search_plain_compareINS_28btree_key_compare_to_adapterIS3_EEEEiRS5_iiRKT_.isra.197.constprop.207>:
  402610:	90                   	nop
  402611:	90                   	nop
  402612:	90                   	nop
  402613:	90                   	nop
  402614:	90                   	nop
  402615:	85 d2                	test   %edx,%edx
  402617:	7e 2e                	jle    402647 <_ZNK15no_unroll_btree10btree_nodeINS_16btree_map_paramsImmSt4lessImESaISt4pairIKmmEELi256EEEE27linear_search_plain_compareINS_28btree_key_compare_to_adapterIS3_EEEEiRS5_iiRKT_.isra.197.constprop.207+0x37>
  402619:	48 8b 0e             	mov    (%rsi),%rcx
  40261c:	48 39 4f 10          	cmp    %rcx,0x10(%rdi)
  402620:	73 25                	jae    402647 <_ZNK15no_unroll_btree10btree_nodeINS_16btree_map_paramsImmSt4lessImESaISt4pairIKmmEELi256EEEE27linear_search_plain_compareINS_28btree_key_compare_to_adapterIS3_EEEEiRS5_iiRKT_.isra.197.constprop.207+0x37>
  402622:	48 83 c7 20          	add    $0x20,%rdi
  402626:	31 c0                	xor    %eax,%eax
  402628:	eb 10                	jmp    40263a <_ZNK15no_unroll_btree10btree_nodeINS_16btree_map_paramsImmSt4lessImESaISt4pairIKmmEELi256EEEE27linear_search_plain_compareINS_28btree_key_compare_to_adapterIS3_EEEEiRS5_iiRKT_.isra.197.constprop.207+0x2a>
  40262a:	66 0f 1f 44 00 00    	nopw   0x0(%rax,%rax,1)
  402630:	48 83 c7 10          	add    $0x10,%rdi
  402634:	48 39 4f f0          	cmp    %rcx,-0x10(%rdi)
  402638:	73 07                	jae    402641 <_ZNK15no_unroll_btree10btree_nodeINS_16btree_map_paramsImmSt4lessImESaISt4pairIKmmEELi256EEEE27linear_search_plain_compareINS_28btree_key_compare_to_adapterIS3_EEEEiRS5_iiRKT_.isra.197.constprop.207+0x31>
  40263a:	83 c0 01             	add    $0x1,%eax
  40263d:	39 c2                	cmp    %eax,%edx
  40263f:	75 ef                	jne    402630 <_ZNK15no_unroll_btree10btree_nodeINS_16btree_map_paramsImmSt4lessImESaISt4pairIKmmEELi256EEEE27linear_search_plain_compareINS_28btree_key_compare_to_adapterIS3_EEEEiRS5_iiRKT_.isra.197.constprop.207+0x20>
  402641:	90                   	nop
  402642:	90                   	nop
  402643:	90                   	nop
  402644:	90                   	nop
  402645:	90                   	nop
  402646:	c3                   	retq
```

The reader will note the code seems smaller; At each loop iteration:

* We add 1 in eax until eax matches edx (0x40263a) and we loop if eax
  does not match edx (0x40263f);
* We add 16 (the size of the tuple "key, value" in the btree) in rdi
  (0x402630) and we exit the loop if the value pointed by rdi (offset:
  -0x10) matches rcx.

## Benchmark

For the whole document, the benchmarked feature will always be the
lookup on a btree lightly loaded (only 64 values; values are 64b
integer); 5000 * 4096 lookups of randomly generated values will be
performed.

Here are the benchmark results for this first version:

* original:  0m0.573s
* no_unroll: 0m0.657s

Relying on the micro-ups cache or the loop buffer instead of gcc
optimizer does not seem a good option.The tested version is nearly 15%
slower.

The reader may conclude that the frontend is not the bottleneck, here;
instruction fetching and decoding are the steps in the pipeline that
the micro-ops cache and the loop buffer can skip.

Let's use perf stat so as to get an idea on the reason why, the
no_unroll version is slower.

Original version:

```
$ perf stat benchmark/bench__lookup google_btree_map

 Performance counter stats for 'benchmark/bench__lookup google_btree_map':

        601.504638      task-clock:u (msec)       #    0.999 CPUs utilized          
                 0      context-switches:u        #    0.000 K/sec                  
                 0      cpu-migrations:u          #    0.000 K/sec                  
               111      page-faults:u             #    0.185 K/sec                  
        1575771503      cycles:u                  #    2.620 GHz                    
        1794679861      instructions:u            #    1.14  insn per cycle         
         706941776      branches:u                # 1175.289 M/sec                  
          41003098      branch-misses:u           #    5.80% of all branches        

       0.602227881 seconds time elapsed
```

No_unroll version:

```
$ perf stat benchmark/bench__lookup no_unroll_btree_map

 Performance counter stats for 'benchmark/bench__lookup no_unroll_btree_map':

        676.003844      task-clock:u (msec)       #    0.999 CPUs utilized          
                 0      context-switches:u        #    0.000 K/sec                  
                 0      cpu-migrations:u          #    0.000 K/sec                  
               109      page-faults:u             #    0.161 K/sec                  
        1790821872      cycles:u                  #    2.649 GHz                    
        2426655448      instructions:u            #    1.36  insn per cycle         
         785767284      branches:u                # 1162.371 M/sec                  
          38434382      branch-misses:u           #    4.89% of all branches        

       0.676612755 seconds time elapsed
```

With the original optimized version, there is no add operation at each
"unrolled" iteration of the loop; thus the count of executed
instructions is significantly reduced.

Saving instructions instead of benefiting from loop buffer or
micro-ops cache seems the winning strategy for this case.

## Conclusion

This first try is as fail at least in the configuration we target
(slow count of keys in the btree container and intensive use of lookup
functionality).

# Trying a first SIMD version

In a btree node, many keys are stored; so, there might be an SIMD
opportunity here: executing the same comparison instruction on many
values (the keys).

Here, we will use some instructions available in the AVX2 set. Instead
of writing inline assembly, we will use intrinsic functions.

## Code

Here is some extract of code so as get an idea on the proposal:

```
int less_uint64_mask4(uint64_t left0,
                      uint64_t left1,
                      uint64_t left2,
                      uint64_t left3,
                      uint64_t right)
{
    // Load the values into ymm registers
    const __m256i values = _mm256_set_epi64x(left3, left2, left1, left0);
    const __m256i targets = _mm256_set1_epi64x(right);

    // Get which ref value is greater than the input value...
    const __m256i result = _mm256_cmpgt_epi64(targets, values);

    // ..and take only the msb per byte coming from the 256b ymm
    // register; so, 256b = 4 x 64b => masking => 32b = 4 x 8b
    const int mask = _mm256_movemask_epi8(result);

    return mask;
}

...

int linear_search_uint64_lower_less_compare(const uint64_t &k,
                                              int s, int e) const {
      
      while (e - s > 4) {

          int mask = less_uint64_mask4(key(s),
                                       key(s + 1),
                                       key(s + 2), key(s + 3), k);

          if (mask != 0xffffffff) {
              mask = ~mask;
              const std::size_t index = __bsfd(mask) / 8;
              return s + index;
          }

          s += 4;
      }
      
      while (s < e) {
        if (!(key(s) < k)) {
        break;
      }
      ++s;
    }

    return s;
  }
```

The comparisons are performed in the first function (less_uint64_mask)
with the instruction avx2 vpcmpgtq (hidden behind the intrinsic
function _mm256_cmpgt_epi64).

## Assembly code

Let's have a look at the assembly code, more precisely the few
instructions before vpcmpgtq (included):

```
  4053d5:	c4 c1 7a 7e 5f 30    	vmovq  0x30(%r15),%xmm3
  4053db:	c4 c1 7a 7e 67 10    	vmovq  0x10(%r15),%xmm4
  4053e1:	c4 c3 e1 22 4f 40 01 	vpinsrq $0x1,0x40(%r15),%xmm3,%xmm1
  4053e8:	c4 c3 d9 22 57 20 01 	vpinsrq $0x1,0x20(%r15),%xmm4,%xmm2
  4053ef:	c4 e2 7d 59 00       	vpbroadcastq (%rax),%ymm0
  4053f4:	c4 e3 6d 38 c9 01    	vinserti128 $0x1,%xmm1,%ymm2,%ymm1
  4053fa:	c4 e2 7d 37 c9       	vpcmpgtq %ymm1,%ymm0,%ymm1
  4053ff:	c5 fd d7 c1          	vpmovmskb %ymm1,%eax
  405403:	83 f8 ff             	cmp    $0xffffffff,%eax
  405406:	0f 85 a4 06 00 00    	jne    405ab0 <_Z20bench_simd_btree_mapv+0x8b0>
  40540c:	83 fe 08             	cmp    $0x8,%esi
  40540f:	0f 8e 3b 05 00 00    	jle    405950 <_Z20bench_simd_btree_mapv+0x750>
  405415:	c4 c1 7a 7e 6f 70    	vmovq  0x70(%r15),%xmm5
  40541b:	c4 c1 7a 7e 77 50    	vmovq  0x50(%r15),%xmm6
  405421:	c4 c3 d1 22 8f 80 00 	vpinsrq $0x1,0x80(%r15),%xmm5,%xmm1
  405428:	00 00 01 
  40542b:	c4 c3 c9 22 57 60 01 	vpinsrq $0x1,0x60(%r15),%xmm6,%xmm2
  405432:	c4 e3 6d 38 c9 01    	vinserti128 $0x1,%xmm1,%ymm2,%ymm1
  405438:	c4 e2 7d 37 c9       	vpcmpgtq %ymm1,%ymm0,%ymm1
```

In order to benefit from SIMD comparison (which is by the way supposed
to work with signed integer values...). we have to fill the ymm
registers; this operation takes 6 other instructions.

So even before benchmarking this new lookup version, the reader can
have a bad feeling on the results.

Once the simd comparison is done, the task is not complete, we need to
find out which compared item matches, if there is one. The idea is to
use a bit bit-scan instruction (forward: bsf/tzcnt) after having
"bit-complemented" a 64bits value (hold in a common register); this
register has been filled with the MSb of each bytes composing the ymm
register (thanks to the intrinsic function _mm256_movemask_epi8 which
holds the instruction vpmovmskb).

Here is the corresponding assembly code:

```
  4053ff:	c5 fd d7 c1          	vpmovmskb %ymm1,%eax
  405403:	83 f8 ff             	cmp    $0xffffffff,%eax
  405406:	0f 85 a4 06 00 00    	jne    405ab0 <_Z20bench_simd_btree_mapv+0x8b0>
...
  405ab0:	f7 d0                	not    %eax
  405ab2:	31 d2                	xor    %edx,%edx
  405ab4:	31 c9                	xor    %ecx,%ecx
  405ab6:	f3 0f bc d0          	tzcnt  %eax,%edx
  405aba:	c1 fa 03             	sar    $0x3,%edx
  405abd:	01 ca                	add    %ecx,%edx
```

Last point to note: the loop is unrolled; so, the assembly blocks
above are repeated many times;

Once more, the code seems larger than the first version. Let's check
whether it has an impact on the performance.

## Benchmark

The same lookup benchmark was applied on this new version, here are
the results:

* original:  0m0.573s
* no_unroll: 0m0.657s
* simd1:     0m0.613s

Even if the "simd1 version is better than the "no_unroll" one, the
original btree code still provides the best time.

Let's have a look at a perf stat report to find out whether the
hypothesis above (too many instructions needed to format ymm
registers) is valid:

```
[alexis@therese build]$ perf stat benchmark/bench__lookup simd_btree_map
Missing lookup: 0

 Performance counter stats for 'benchmark/bench__lookup simd_btree_map':

        639.710235      task-clock:u (msec)       #    0.999 CPUs utilized          
                 0      context-switches:u        #    0.000 K/sec                  
                 0      cpu-migrations:u          #    0.000 K/sec                  
               108      page-faults:u             #    0.169 K/sec                  
        1671889896      cycles:u                  #    2.614 GHz                    
        1923975547      instructions:u            #    1.15  insn per cycle         
         493786094      branches:u                #  771.890 M/sec                  
          38437839      branch-misses:u           #    7.78% of all branches        

       0.640356910 seconds time elapsed
```

In this version, less instructions were executed than in the
unroll_version but it is still more than in the original version. As
the count of executed instructions is a pretty good indicator
(considering the fact that the instructions per cycle does not change
a lot), let's make a little summary here:

* original:  1794679861 instructions #  1.14  insn per cycle         
* no_unroll: 2426655448 instructions #  1.36  insn per cycle         
* simd1:     1923975547 instructions #  1.15  insn per cycle         

The reader can at least conclude that even if using the avx2
comparison instruction needs many "registers preparation"
instructions, simd can reduce significantly the count of executing
instructions.


# Trying a second simd version

Here, let's try to reduce the count "ymm registers preparation
instructions" with the new avx2 gather instruction, which fills a ymm
register thanks uncontiguous memory loads.

## Code

```
  int linear_search_uint64_lower_less_compare(const uint64_t &k,
                                              int s, int e) const {

      static const std::size_t step = kValueSize / sizeof(uint64_t);
      
      const long long int* base =
          reinterpret_cast<const long long int*>(fields_.values);
      const __m256i rights = _mm256_set1_epi64x(k);
      
      while (e - s > 4) {

          const __m256i indices =
              _mm256_set_epi64x((s + 3) * step,
                                (s + 2) * step, (s + 1) * step, s * step);          
          
          const __m256i lefts =
              _mm256_i64gather_epi64(base, indices, sizeof(uint64_t));
          
          int mask = less_uint64_mask4(lefts, rights);

          if (mask != 0xffffffff) {
              mask = ~mask;
              const std::size_t index = __bsfd(mask) / 8;
              return s + index;
          }

          s += 4;
      }
      
      while (s < e) {
        if (!(key(s) < k)) {
        break;
      }
      ++s;
    }

    return s;
  }
```

The change, here, is the use of the intrinsic function
_mm256_i64gather_epi64 which hides the instruction vpgatherqq.

## Assembly code

Once more, let's have a look at the instructions around vpcmpgtq:

```
  405d00:	83 c1 04             	add    $0x4,%ecx
  405d03:	48 83 c2 08          	add    $0x8,%rdx
  405d07:	39 f1                	cmp    %esi,%ecx
  405d09:	0f 84 d9 01 00 00    	je     405ee8 <_Z21bench_simd2_btree_mapv+0x398>
  405d0f:	48 8d 42 04          	lea    0x4(%rdx),%rax
  405d13:	4c 8d 4a 02          	lea    0x2(%rdx),%r9
  405d17:	c4 e1 f9 6e e2       	vmovq  %rdx,%xmm4
  405d1c:	c4 e1 f9 6e c0       	vmovq  %rax,%xmm0
  405d21:	48 8d 42 06          	lea    0x6(%rdx),%rax
  405d25:	c4 c3 d9 22 c9 01    	vpinsrq $0x1,%r9,%xmm4,%xmm1
  405d2b:	c4 e3 f9 22 c0 01    	vpinsrq $0x1,%rax,%xmm0,%xmm0
  405d31:	c5 fd 6f eb          	vmovdqa %ymm3,%ymm5
  405d35:	c4 e3 75 38 c0 01    	vinserti128 $0x1,%xmm0,%ymm1,%ymm0
  405d3b:	c4 c2 d5 91 0c c0    	vpgatherqq %ymm5,(%r8,%ymm0,8),%ymm1
  405d41:	c4 e2 6d 37 c1       	vpcmpgtq %ymm1,%ymm2,%ymm0
  405d46:	c5 fd d7 c0          	vpmovmskb %ymm0,%eax
  405d4a:	83 f8 ff             	cmp    $0xffffffff,%eax
  405d4d:	74 b1                	je     405d00 <_Z21bench_simd2_btree_mapv+0x1b0>
```

The first point to note, here, is that the loop was not unrolled;
after having executing the SIMD comparison instruction (vpcmpgtq) and
the masking one, we check whether the result value is worth
0xffffffff. We saw in the first test "no_unroll" that unrolling has
bad consequences on the performance.

The reason why the compiler decided not to unroll may be because there
are too many instructions per iteration. If this hypothesis is true,
it means that I did not manage to reduce the "prologue" code before
"vpcmpgtq"... which is true if we count the instructions.

As a consequence, the expectations in this second SIMD version are not
high; let's check whehter these assumptions are right.

## Benchmark

Once more, the same lookup benchmark was applied on this version, here
are the results:

* original:  0m0.573s
* no_unroll: 0m0.657s
* simd1:     0m0.613s
* simd2:     0m0.733s

As expected, the results are disappointing; the "simd2" time is even
higher than the "no_unroll" version.

The "perf stat" output below seems to enforce the feeling that
performance loss is due to a too high count of executed instructions.

```
$ perf stat benchmark/bench__lookup simd2_btree_map

 Performance counter stats for 'benchmark/bench__lookup simd2_btree_map':

        758.998569      task-clock:u (msec)       #    0.999 CPUs utilized          
                 0      context-switches:u        #    0.000 K/sec                  
                 0      cpu-migrations:u          #    0.000 K/sec                  
               110      page-faults:u             #    0.145 K/sec                  
        2012908504      cycles:u                  #    2.652 GHz                    
        2328906873      instructions:u            #    1.16  insn per cycle         
         476431169      branches:u                #  627.710 M/sec                  
          38086999      branch-misses:u           #    7.99% of all branches        

       0.759611490 seconds time elapsed
```

# Conclusion

So far, I was unable to improve lookup performance of the google btree
with AVX2 SIMD instructions. It will be difficult to be clearer...

There is, however, a last point which might be interesting to check:
if loop unrolling is disabled in "simd1" version, wihch version is the
best between "simd1(no_unroll)" and "simd2(always_no_unroll)".